In [1]:

from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
seed_everything(1376)

Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [2]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [13]:
configs

Namespace(is_baseline=False, model_base='facebook/bart-base', output_dir='trained_models_sum_boundary/', task_type='xsum', run_id='bart_base_model_context_enforcer_250', eval_steps=1000, learning_rate=5e-05, max_seq_len=800, evaluation_strategy='epoch', save_strategy='epoch', seed=10, lr_scheduler_type='linear', weight_decay=0.0, warmup_ratio=0.25, num_train_epochs=10, save_total_limit=1, per_device_train_batch_size=12, per_device_eval_batch_size=12, gradient_accumulation_steps=4, fp16=True, verbose=False, context_max_len=250)

In [2]:
configs= pk.load(open("trained_models_sum_boundary/bart_base_model_context_enforcer_v2_250/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_sampling_bounds=(0.1, 0.45)

In [18]:
configs.task_type

'xsum'

In [3]:
context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
generator = model_builder()
train_model_path = "trained_models_sum_boundary/bart_base_model_context_enforcer_v2_250/checkpoint-25506/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of BartForContextualRecovery were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.layers.5.context_enforcer._right_mha.in_proj_weight', 'encoder.layers.0.context_enforcer._wlc.weight', 'encoder.layers.5.context_enforcer._wlc.bias', 'encoder.layers.4.context_enforcer._wlc.weight', 'encoder.layers.3.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.2.context_enforcer._left_mha.out_proj.bias', 'encoder.layers.5.context_enforcer._wrc.bias', 'encoder.layers.3.context_enforcer._wrc.weight', 'encoder.layers.2.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.0.context_enforcer._right_mha.in_proj_weight', 'encoder.layers.1.context_enforcer._wlc.weight', 'encoder.layers.4.context_enforcer_layer_norm.bias', 'encoder.layers.3.context_enforcer._left_mha.out_proj.weight', 'encoder.layers.4.context_enforcer_layer_norm.weight', 'encoder.layers.1.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.0.conte

In [8]:
te1= test_dataset[6]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
seed_everything(1376)
boundary_sample =  (0.1, 0.45)
seq_len = te1.input_ids[:1024].shape[0]

boundary_start = int(0.45*seq_len) 
context_boundary = compute_context_boundary(seq_len,context_max_len=context_max_len)
context_boundary

Global seed set to 1376


(264, 514)

In [10]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=(10,200),
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=129,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['Double Paralympic champion Wendy Houvenaghel has said she felt "vindicated" by British Cycling\'s World Class programme.']

In [8]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'A "medal at any cost" approach created a "culture of fear" at British Cycling, says former rider Wendy Houvenaghel.'

In [51]:
te1.input_ids.shape

torch.Size([1187])

In [58]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

Media playback is not supported on this device
The Olympic silver medallist accused the organisation of "ageism" and having "zero regard" for her welfare.
She is the latest high-profile cyclist to come forward after Jess Varnish, Nicole Cooke and Emma Pooley criticised the World Class programme.
Houvenaghel told the BBC she felt "vindicated" by a leaked draft report detailing British Cycling's failures.
The report said British Cycling "sanitised" its own investigation into claims former technical director Shane Sutton used sexist language towards Varnish, who went public last April about her treatment.
British Cycling subsequently admitted it did not pay "sufficient care and attention" to the wellbeing of staff and athletes at the expense of winning medals, an approach Houvenaghel attested to in her BBC interview.
Both Sutton and predecessor Sir Dave Brailsford have now left British Cycling.
Houvenaghel, 42, spoke to BBC Sport during its State of Sport week, which on Thursday examines 

In [31]:
boundary_mask

(360, 610)

In [11]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
test_data_loader = DataLoader(test_dataset,batch_size=12,
                              sampler= SequentialSampler(test_dataset),
                              collate_fn= SmartCollator(
            pad_token_id=train_dataset.tokenizer.pad_token_id,
            max_len=configs.max_seq_len,
            context_max_len=context_max_len,
            context_sampling_bounds=context_sampling_bounds,
           
        )
                              )

output_summaries2 =[]
for batch in tqdm.tqdm(test_data_loader):
    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    
    seq_len = b_input_ids.shape[1]

    context_boundary = compute_context_boundary(seq_len,
                                                context_max_len=110)
    
    boundary_mask =  batch.get("boundary",)
    bb=generator.generate(input_ids=b_input_ids,
            attention_mask=b_input_mask,
            #context_boundary=boundary_mask,
            num_beams=10,
            do_sample=False,
            num_return_sequences=1,
            max_length=140)
    sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
    output_summaries2+=sentences

100%|██████████| 945/945 [08:35<00:00,  1.83it/s]


In [13]:
os.makedirs('outputs/attempt2/')

In [14]:
#context_enforcement/data/common.py
from context_enforcement.data.common import write_to_file
write_to_file(output_summaries2[:len(test_dataset)], "outputs/attempt2/simple_new2")

Done


In [15]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

In [16]:
targets[690],output_summaries2[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 "Uganda's Koffi Olomide has been arrested in the Democratic Republic of Congo on suspicion of assault, his lawyer says.")

In [17]:
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])


[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
scores = metrics.compute(predictions=output_summaries2,references=targets)

In [34]:
scores4 = metrics.compute(predictions=output_summaries2,references=targets)

In [34]:
scores

{'bleu': 0.1141433732377109,
 'precisions': [0.43245642095398573,
  0.17026004301606218,
  0.09024261735705146,
  0.051603875478183146],
 'brevity_penalty': 0.8388096738071763,
 'length_ratio': 0.8505054298261077,
 'translation_length': 221953,
 'reference_length': 260966,
 'meteor': 0.3387483983011999,
 'rouge1': 0.390985647831118,
 'rouge2': 0.1734127161917697,
 'rougeL': 0.32029615648919607,
 'rougeLsum': 0.32020127586684505}

In [19]:
scores

{'bleu': 0.11674485337622066,
 'precisions': [0.42922052452745374,
  0.16942131508755653,
  0.08995745222555707,
  0.05176069297640832],
 'brevity_penalty': 0.8606296507042277,
 'length_ratio': 0.869496409493957,
 'translation_length': 226909,
 'reference_length': 260966,
 'meteor': 0.34353560641358893,
 'rouge1': 0.3932087312039648,
 'rouge2': 0.1753259888541615,
 'rougeL': 0.320916661600919,
 'rougeLsum': 0.3208579853859774}

In [28]:
scores3

{'bleu': 0.11312982003210258,
 'precisions': [0.43367935409457903,
  0.170965703452634,
  0.09041255378385218,
  0.051220088499377064],
 'brevity_penalty': 0.8310753377221631,
 'length_ratio': 0.8438570541756397,
 'translation_length': 220218,
 'reference_length': 260966,
 'meteor': 0.3379248069191188,
 'rouge1': 0.3902876664001791,
 'rouge2': 0.17375564911724187,
 'rougeL': 0.3204999917403623,
 'rougeLsum': 0.3205687202163906}

In [35]:
scores4

{'bleu': 0.11146082917260128,
 'precisions': [0.4317755562839142,
  0.1685194251648763,
  0.08904895332074475,
  0.050525125222168364],
 'brevity_penalty': 0.8286310630214466,
 'length_ratio': 0.8417648276020631,
 'translation_length': 219672,
 'reference_length': 260966,
 'meteor': 0.334512237524537,
 'rouge1': 0.3879467178046803,
 'rouge2': 0.17119907794549927,
 'rougeL': 0.31801217096792334,
 'rougeLsum': 0.31809606706132665}